<a href="https://colab.research.google.com/github/davidkant/mai/blob/master/tutorial/5_2_Evolved_FM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5.2 Evolved FM

The course package `mai` provides the skeleton of GA loaded with default functions, but in order to do anything particulalrly useful with it, we need to customize a few of these functions to suit our particular application. This notebook shows how to custom the default `GeneticAlgorithm()` to evolve some electronically synthesized sounds using FM synthesis. 

## Setup
Install external libraries and import them into your notebook session.

In [0]:
# install external libraries for sound playback
from IPython.display import clear_output
!pip install -q git+https://github.com/davidkant/mai#egg=mai;
!apt-get -qq update
!apt-get -qq install -y libfluidsynth1
clear_output()

In [0]:
# imports
import mai
import mai.synths
import mai.genalg
import mai.musifuncs as mf
import IPython.display
import time
import matplotlib.pyplot as plt
import random

Using TensorFlow backend.


## About Frequency Modulation (FM) Synthesis

In this notebook we'll use a form of digital sound synthesis call *frequency modulation* (FM). We're using FM synthesis beacuse, even though it is capable of producing a wide variety of sounds through different combinations of parameter settings, changing a given parameter can affect the sound in complex and unpredictable ways. Check out [Music and Computers: Chapter 4](http://sites.music.columbia.edu/cmc/MusicAndComputers/chapter4/04_07.php) if you'd like to learn more about FM synthesis. 

Our FM synthesizer has 6 parameters that determine the resulting sounds:
* carrier frequency (in hertz)
* modulator frequency (in hertz)
* index start
* index end
* attack (in seconds)
* release (in seconds)





The course code package includes a function `synths.fm()` to implement FM synthesis. We'll specify our paramters as a list of numbers and pass that list to the function. Here's an example of an FM synth with parameters: *carrier frequency*: 700 Hz; *modulator frequency*: 300 Hz; *index start*: 5; *index end*: 0, *attack*: 0; *release* 0.333. Try changing a few values — can you develop of a sense of how changing the parameters will affect the sound?

In [0]:
# synthesize audio using FM synthesis
y = mai.synths.fm(carrier=700, modulator=300, index1=5, index2=10, attack=2, release=0.33)

# play it
IPython.display.Audio(y, rate=44100, autoplay=False)

# Part 1: Create a new GA

In this section we'll extend the default `GeneticAlgorithm()` model to fit our FM synth. First, create a default `GeneticAlgothim()`. We'll override the defaults functions `random_individual()` and `to_phenotype()` with our own versions next.


In [0]:
# create a genetic algorithm object
ga = mai.genalg.GeneticAlgorithm()

### genotype
Since our FM synth has 6 parameters, our genotype will be a list of 6 random numbers between 0 and 1. In the cell below we define a new function for `random_individual()` that generates a list of 6 random numbers between 0 and 1. In the last line of the cell, we overwrite the default `random_individual` function of our GA with our new one. Note that we used `random.random()` this time to generate a floating-point random number between `0.0` and `1.0`.

In [0]:
def random_individual():
  """Generate random genotype 6 values in range [0,1]."""
  
  # create a random genotype
  genotype = [random.random() for i in range(6)]

  # return it
  return genotype

# overwrite default function 
ga.random_individual = random_individual

### phenotype
Here we write a new `to_phenotype` function. The function first scales the genotype values to appropriate ranges for the synthesizer, and then passes the scaled parameter values to `synths.FM1()` to synthesize the sound. In the last line of the cell, we overwrite the default `to_phenotype` function of our GA with our new one. We're using a new function `mf.exp_map` to scale the genotype values along an exponential scale.

In [0]:
def to_phenotype(genotype):
  """Convert genotype to sound using FM synthesis."""
    
  # scale values
  carrier   = mf.scale(genotype[0], 1, 10000, kind='exp')  # carrier freq
  modulator = mf.scale(genotype[1], 1, 10000, kind='exp')  # modulator freq 
  index1    = mf.scale(genotype[2], 1, 100, kind='exp')    # index start
  index2    = mf.scale(genotype[3], 1, 100, kind='exp')    # index end
  attack    = mf.scale(genotype[4], 0.01, 5, kind='exp')   # attack
  release   = mf.scale(genotype[5], 0.01, 5, kind='exp')   # release
  
  # synthesize audio using FM synthesis
  y = mai.synths.fm(carrier, modulator, index1, index2, attack, release)
  
  return y

# overwrite default function 
ga.to_phenotype = to_phenotype

We're now setup to evolve some sounds using FM synthesis!

# Part 2: Use the Genetic Algorithm to Evolve Sounds
Now that we're all setup, we can use our custom GA to evolve the sounds of FM synthesis. We will initialize a random population, listen, assign fitness scores, evolve the next generation, and repeat!

## Step 1: Initialize random population
First we construct an initial random population. Note: you can change the size of the population here by adjusting the value of the argument `population_size`.

In [0]:
# initialize random population
ga.initialize_population(population_size=12)

## Step 2: Convert genotype to phenotype and listen
Now we want to listen to each individual and evaluate its fitness. First we have to convert each genotype to its corresponding phenoype  and then play it. The cell below loops through  each of the twelve individuals, converts to phenotype, and plays it back.

In [0]:
# loop through entire popularion
for i,genotype in enumerate(ga.population):

  print('sounding individual {0}'.format(i))
    
  # convert to phenotype
  y = ga.to_phenotype(genotype)
    
  # play it
  d = IPython.display.Audio(y, rate=44100, autoplay=False)
  IPython.display.display(d)
   
  # wait before moving to next
  # time.sleep(y.shape[0]/44100.0 + 1.0)

sounding individual 0


sounding individual 1


sounding individual 2


sounding individual 3


sounding individual 4


...

sounding individual 7


sounding individual 8


sounding individual 9


sounding individual 10


sounding individual 11


You can view the genotype of a particular individual by changing the index of `ga.population[0]`

In [0]:
ga.population[0]

[0.7421952193303252,
 0.13474831454018654,
 0.09672119881393293,
 0.12615722750565161,
 0.5267259349320329,
 0.9357780453011112]

## Step 3: Set fitness scores
As you listen to each individual decide on a fitness score. Remember, this must be a value between 0 and 1! Set the fitness scores for each of the twelve individuals in the cell below by chaning the values to the right of the equals sign `=`. Individuals are indexed by the number in the first set of square brackets `[]`, so `ga.fitness[0][0]` refers to indivudal 1, `ga.fitness[1][0]` refers to individul 2, and so on up to `ga.fitness[11][0]` refers to individual 12.

In [0]:
# set fitness scores
ga.fitness[0][0] = 0
ga.fitness[1][0] = 0
ga.fitness[2][0] = 0
ga.fitness[3][0] = 0
ga.fitness[4][0] = 0
ga.fitness[5][0] = 0
ga.fitness[6][0] = 0
ga.fitness[7][0] = 1
ga.fitness[8][0] = 0
ga.fitness[9][0] = 1
ga.fitness[10][0] = 0
ga.fitness[11][0] = 0

## Step 4. Evolve the next generation
Once the fitness scores are set you can evolve the next generation. Note: you can adjust the mutation probability here by changing the value of the argument `mutation_prob`.

In [0]:
# evolve next generation
ga.evolve_once(mutation_prob=0.1)

## Repeat! 
Continue to evolve generations by looping back to **Step 2** and repeating. **Note:** repeats **Steps 2-4** to evolve new generations that build on the previous. Loop all the way back to **Step 1** to initialize a new population when you are ready to start a new search.